In [9]:
from textblob_de import TextBlobDE as TextBlob

In [10]:
text = "Heute ist der 3. Mai 2014 und Dr. Meier feiert seinen 43. Geburtstag. Verdammte Scheiße! Das ist so blöd. Ich muss unbedingt daran denken, Mehl, usw. für einen Kuchen einzukaufen. Aber leider habe ich nur noch EUR 3.50 in meiner Brieftasche."
blob = TextBlob(text)

In [11]:
blob.sentences

[Sentence("Heute ist der 3. Mai 2014 und Dr. Meier feiert seinen 43. Geburtstag."),
 Sentence("Verdammte Scheiße!"),
 Sentence("Das ist so blöd."),
 Sentence("Ich muss unbedingt daran denken, Mehl, usw. für einen Kuchen einzukaufen."),
 Sentence("Aber leider habe ich nur noch EUR 3.50 in meiner Brieftasche.")]

In [12]:
blob.tokens

WordList(['Heute', 'ist', 'der', '3.', 'Mai', '2014', 'und', 'Dr.', 'Meier', 'feiert', 'seinen', '43.', 'Geburtstag', '.', 'Verdammte', 'Scheiße', '!', 'Das', 'ist', 'so', 'blöd', '.', 'Ich', 'muss', 'unbedingt', 'daran', 'denken', ',', 'Mehl', ',', 'usw.', 'für', 'einen', 'Kuchen', 'einzukaufen', '.', 'Aber', 'leider', 'habe', 'ich', 'nur', 'noch', 'EUR', '3.50', 'in', 'meiner', 'Brieftasche', '.'])

In [13]:
TextBlob(text).sentiment

Sentiment(polarity=-0.2, subjectivity=0.0)

In [14]:
import pandas as pd

In [15]:
ruegi_comments = pd.read_csv("ruegi_comments_new_pipeline_STEMMED_VERSION.csv")
ruegi_comments = ruegi_comments.drop(["Unnamed: 0"], axis=1)
fresh_copy = ruegi_comments.copy()
fresh_copy["comments"] = fresh_copy["comments"].astype(str)
fresh_copy

fresh_copy = fresh_copy.assign(blob_prediction = fresh_copy["comments"].apply(lambda row: TextBlob(row).sentiment[0]))

In [16]:
fresh_copy.to_csv("ruegie_comments_with_blob_predictions_STEMMED_VERSION.csv")

### Zwischenergebnis

TextBlob und SVC entscheiden in vielen Fällen der binären Klassifizierung ähnlich, wenn man alle Werte > 0 als positiv und alle Werte < 0 als negativ wertet. Die genauere Einschätzung von TextBlob mit einer Spanne von -1 bis 1 (genannt "Polarität") ist nur in wenigen Fällen mit dem vorliegenden Datensatz akkurat, jedoch zeigen sich zu diesem Zeitpunkt an bestimmten Stellen Vorteile gegenüber des rein binären SVC:

In Reihe 67 von fresh_copy lautet der Kommentar "Lächerlich". Auch für binäre Modelle eine eigentlich eindeutig negative Wertung. Das SVC kategorisiert dieses Wort jedoch in "positiv == 1". Wie ist das zu erklären? Im Text-Preprocessing wurden einzelne Wörter mithilfe eines Stemmers verkürzt. Das heißt, wiederkehrende Wortendungen wurden abgeschnitten, um die Vergleichbarkeit verschiedener Sätze zu erhöhen. Außerdem wurden die deutschen Umlaute in die ihnen zugrundeliegenden Vokale umgewandelt. Im Endergebnis wird also "Lächerlich" zu "Lach -er -lich" und schließlich zu "Lach". Zwar wird die erste Person Präsens von "lachen" häufig in einem ironischen Kontext gebraucht. Genauso würde jedoch "lachen" selbst in jedem einzelnen Fall zu "lach -en" und zu "lach" werden, vermutlich eingesetzt in einem mehrheitlich positiven Kontext. So kommt es in diesem Fall zu einer gravierenden Fehleinschätzung der Support-Vector-Machine.
TextBlob hingegen erkennt mit einer Polarität von -1 sicher den negativen Charakter des Ausdrucks.
An diesem Beispiel zeigen sich die vielen Fallstricke vermeintlich simpler Klassifizierungsmodelle. Eine Lemmatisierung der einzelnen Wörter eines Satzes führt in solchen Fällen zu besseren Ergebnissen, da die Satzbausteine lediglich in ihre Grundform zurückgeführt werden. "Ich lache" würde also in mehreren Schritten zu "\[ich\], \[lachen\]" oder "Das ist lächerlich" zu "\[das\], \[sein\], \[lächerlich\]". Es zeigt sich allerdings, dass die Lemmatisierung (welche vermutlich von TextBlob eingesetzt wird) in anderen Fällen zu deutlich schlechterer Vergleichbarkeit eigentlich ähnlicher Aussagen führt und die Komplexität des Vokabulars der Klassifizierungsmodelle erhöht.